Import functions, setup list of recordings for iteration

In [5]:
# setup
import gridtools as gt
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import numpy as np
import pandas as pd
%matplotlib qt

from plotstyle import PlotStyle
s = PlotStyle()

In [6]:
# import data from export folder
path = "../processed_data/"

# import df resting and df event
deltafs = pd.read_csv(path+"diff_f_before_vs_during_event.csv")
fincreases = pd.read_csv(path+"event_f_increase.csv")

# import kernel density estimates
kde_dist_all = pd.read_csv(path+"kde_d_all.csv")
kde_dist_interact = pd.read_csv(path+"kde_d_interact.csv")
kde_dist_events = pd.read_csv(path+"kde_d_events.csv")
bootstrap_kde = pd.read_csv(path+"kde_bootstrap.csv")

# import raster data
raster_ts_init = np.load(path+"raster_ts_init.npy", allow_pickle=True)
raster_ts_react = np.load(path+"raster_ts_react.npy", allow_pickle=True)

In [7]:
def kde_hide(ax):
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.spines['top'].set_visible(False)

from plotstyle import PlotStyle
s = PlotStyle()

# Grid setup
hr = [1, 1]
wr = [0.5, 0.01, 0.5, 1]
ny = len(hr)
nx = len(wr)
fig = plt.figure(figsize=(300 * s.mm, 130 * s.mm), constrained_layout=False)

grid = GridSpec(ny, nx, figure=fig, height_ratios=hr, width_ratios=wr)

grid.update(left=0.06, right=0.98, bottom=0.12, top=0.91, wspace=0.2, hspace=0.7)

ax1 = fig.add_subplot(grid[:, 0])
ax2 = fig.add_subplot(grid[:, 2])
ax3 = fig.add_subplot(grid[0, 3])
ax4 = fig.add_subplot(grid[1, 3])

# ------------------------------------------------------------------------------
# plot case against jamming avoidance 
# ------------------------------------------------------------------------------

height1 = 0  # x where initiator points go
height2 = 0.2  # x where reactor points go
bp_width = 0.02 # boxplot width
bp_offset = 0.03  # offset for boxplots
rc_offset = 0.06  # offset for kde
jitter_width = 0.002 # width of jittered data

# make jitter, plot scatter
jit_x1 = np.random.normal(height1, jitter_width, size=len(deltafs["before"]))
jit_x2 = np.random.normal(height2, jitter_width, size=len(deltafs["during"]))

# make connectors
for res, eve, x1, x2 in zip(deltafs["before"], deltafs["during"], jit_x1, jit_x2):
    y_tmp = [res, eve]
    x_tmp = [x1, x2]
    ax1.plot(x_tmp, y_tmp, color="black", linewidth=1, alpha=0.3, zorder = -2000)

y_tmp = [np.mean(deltafs["before"]), np.mean(deltafs["during"])]
ax1.plot(x_tmp, y_tmp, color="black", linewidth=1.5, alpha=0.8)

ax1.plot(
    jit_x1,
    deltafs["before"],
    **s.scatter1
)
ax1.plot(
    jit_x2,
    deltafs["during"],
    **s.scatter2
)
# plot boxplots
bp = ax1.boxplot(
    deltafs["before"],
    positions=np.array([height1 - bp_offset]),
    widths=np.array([bp_width]),
    manage_ticks=False,
)
s.set_boxplot_color(bp, "black")

bp = ax1.boxplot(
    deltafs["during"],
    positions=np.array([height2 + bp_offset]),
    widths=np.array([bp_width]),
    manage_ticks=False,
)
s.set_boxplot_color(bp, "black")

# plot kdes
lims_df = gt.utils.lims(deltafs["before"], deltafs["during"])
lims_df = [0, lims_df[1] + 5]

kde_diff_resting = gt.utils.kde1d(deltafs["before"], 2, lims_df)
kde_diff_event = gt.utils.kde1d(deltafs["during"], 2, lims_df)

ax1.fill_betweenx(
    kde_diff_resting[0],
    np.full_like(kde_diff_resting[1], height1 - rc_offset),
    kde_diff_resting[1] * (-1) + height1 - rc_offset,
    **s.kde1,
    lw=0,
)
ax1.fill_betweenx(
    kde_diff_event[0],
    np.full_like(kde_diff_event[1], height2 + rc_offset),
    kde_diff_event[1] + height2 + rc_offset,
    **s.kde2,
    lw=0,
)
ax1.set_ylim(0, None)

ax1.set_xlim(-0.20, 0.40)
x_ticks = [0, 0.2]
x_labels = ["resting", "event"]
ax1.set_xticks(ticks=x_ticks, labels=x_labels)
ax1.set_ylabel("$\Delta$ EOD$f$  [Hz]")
ax1.set_xlabel("time")
ax1.spines['right'].set_visible(False)
ax1.spines['top'].set_visible(False)
ax1.spines['bottom'].set_visible(False)

# ------------------------------------------------------------------------------
# Plot initiator vs reactor frequency increase
# ------------------------------------------------------------------------------

height1 = 0  # x where initiator points go
height2 = 0.2  # x where reactor points go
bp_width = 0.02 # boxplot width
bp_offset = 0.03  # offset for boxplots
rc_offset = 0.06  # offset for kde
jitter_width = 0.002 # width of jittered data

# plot scatterplot with jitter
jit_x1 = np.random.normal(height1, jitter_width, size=len(fincreases["init"]))
jit_x2 = np.random.normal(height2, jitter_width, size=len(fincreases["react"]))


# plot scatterplot connectors
for init, react, x1, x2 in zip(fincreases["init"], fincreases["react"], jit_x1, jit_x2):
    y_tmp = [init, react]
    x_tmp = [x1, x2]
    ax2.plot(x_tmp, y_tmp, color="black", linewidth=1, alpha=0.3, zorder = -2000)
y_tmp = [np.mean(fincreases["init"]), np.mean(fincreases["react"])]
ax2.plot(x_tmp, y_tmp, color="black", linewidth=1.5, alpha=0.8)

ax2.plot(
    jit_x1,
    fincreases["init"],
    **s.scatter1
)
ax2.plot(
    jit_x2,
    fincreases["react"],
    **s.scatter2
)

# plot boxplots
bp1 = ax2.boxplot(
    fincreases["init"],
    positions=np.array([height1 - bp_offset]),
    widths=np.array([bp_width]),
    manage_ticks=False,
)
s.set_boxplot_color(bp1, "black")
bp2 = ax2.boxplot(
    fincreases["react"],
    positions=np.array([height2 + bp_offset]),
    widths=np.array([bp_width]),
    manage_ticks=False,
)
s.set_boxplot_color(bp2, "black")

# plot kde
xlims = [0, 50]
kde_init_amp = gt.utils.kde1d(fincreases["init"], 1, xlims)
kde_react_amp = gt.utils.kde1d(fincreases["react"], 1, xlims)

ax2.fill_betweenx(
    kde_init_amp[0],
    np.full_like(kde_init_amp[1], height1 - rc_offset),
    kde_init_amp[1] * (-1) + height1 - rc_offset,
    **s.kde1,
    lw=0,
)

# plot kde
ax2.fill_betweenx(
    kde_react_amp[0],
    np.full_like(kde_react_amp[1], height2 + rc_offset),
    kde_react_amp[1] + height2 + rc_offset,
    **s.kde2,
    lw=0,
)
ax2.set_xlim(-0.18, 0.38)
# specify x-axis locations
x_ticks = [0, 0.2]
# specify x-axis labels
x_labels = ["init", "react"]
# add x-axis values to plot
ax2.set_xticks(ticks=x_ticks, labels=x_labels)
ax2.set_ylabel("EOD$f$ up-modulation [Hz]")
ax2.set_xlabel("interaction role")
ax2.spines['right'].set_visible(False)
ax2.spines['top'].set_visible(False)
ax2.spines['bottom'].set_visible(False)

# ------------------------------------------------------------------------------
# Distance kernel desity estimates
# ------------------------------------------------------------------------------

ax3.fill_between(
    kde_dist_all['x'], 
    kde_dist_all['y'], 
    label="all individuals",
    **s.kde2,
    lw=1.5,
)
ax3.fill_between(
    kde_dist_interact["x"],
    kde_dist_interact["y"],
    label="interact. individuals",
    **s.kde3,
    lw=1.5,
)
ax3.fill_between(
    kde_dist_events["x"],
    kde_dist_events["y"],
    label="sync. modulations",
     **s.kde1,
     lw=1.5,
)
ax3.set_xlim(-10, None)
ax3.set_xlabel("distance [cm]")
ax3.set(yticklabels=[])
ax3.tick_params(left=False) 
ax3.set_xlim(0,400)
yscaler = 0.1*np.max(kde_dist_events["y"])
ax3.set_ylim(0-yscaler, np.max(kde_dist_events["y"]) + yscaler)
kde_hide(ax3)

ax3.legend(
    facecolor = 'white',
    columnspacing = 1,
    framealpha = 0.6,
    bbox_to_anchor=(1, 0.6),
    loc="lower right",
    ncol=1,
    handlelength=1,
)

# ------------------------------------------------------------------------------
# Event triggered interactions
# ------------------------------------------------------------------------------

ax4.eventplot(raster_ts_init, linelengths=0.85,color = "lightgrey",  alpha = 1, zorder = 1000)
ax4.eventplot(raster_ts_react, linelengths=0.85, color = "lightgrey", alpha = 1, zorder = 1000)

ax5 = ax4.twinx()

# plot kde for initiator and reactor

# ax5.fill_between(
#     kde_raster_init["x"], 
#     kde_raster_init["y"], 
#     label="initiator",
#     **s.kde1,
#     )

# ax5.fill_between(
#     kde_raster_react["x"], 
#     kde_raster_react["y"], 
#     label="reactor",
#     **s.kde2,
#     )

# plot kde for all instead

ax5.fill_between(
    bootstrap_kde["x"], 
    bootstrap_kde["estimated_q5"],
    bootstrap_kde["estimated_q95"], 
    **s.kde1_line_error,
    )

ax5.plot(
    bootstrap_kde["x"], 
    bootstrap_kde["estimated_median"], 
    **s.kde1_line,
    label = 'kernel density',
    zorder = 1002,
    )

ax5.fill_between(
    bootstrap_kde["x"], 
    bootstrap_kde["baseline_q5"],
    bootstrap_kde["baseline_q95"], 
    **s.kde2_line_error,
    )

ax5.plot(
    bootstrap_kde["x"], 
    bootstrap_kde["baseline_median"], 
    **s.kde2_line,
    label = 'permutation',
    )

ax5.axvline(0, lw=1, ls="dashed", color="black")

ax4.set(yticklabels=[])
ax4.tick_params(left=False)
ax4.set_xlabel("sync. mod. centered time [s]")
kde_hide(ax4)
ax4.set_ylim(-1.5, None)
ax4.set_xlim(-300, 600)

ax5.tick_params(right=False)
ax5.set(yticklabels=[])
kde_hide(ax5)
yscaler = 0.1*np.max(bootstrap_kde["estimated_q95"])
ax5.set_ylim(0-yscaler, np.max(bootstrap_kde["estimated_q95"]) + yscaler)
ax5.set_xlim(-300, 600)

ax5.legend(
    facecolor = 'white',
    columnspacing = 1,
    framealpha = 0.6,
    bbox_to_anchor=(1, 0.8),
    loc="lower right",
    ncol=1,
    handlelength=1,
)


# labels for subplots

xp1, yp1 = -0.25, 1.05
xp2, yp2 = -0.05, 1.147

ax1.text(xp1, yp1, 'A', transform=ax1.transAxes, size=16, weight = 'bold')
ax2.text(xp1, yp1, 'B', transform=ax2.transAxes, size=16, weight = 'bold')
ax3.text(xp2, yp2, 'C', transform=ax3.transAxes, size=16, weight = 'bold')
ax5.text(xp2, yp2, 'D', transform=ax5.transAxes, size=16, weight = 'bold')



fig.savefig('../poster/figs/eventstats.pdf')

/home/weygoldt/Data/uni/efish/env/lib/python3.10/site-packages/gridtools/functions.py:107: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  kde = KernelDensity(kernel=kernel, bandwidth=bandwidth).fit(y[:, None])
/home/weygoldt/Data/uni/efish/env/lib/python3.10/site-packages/gridtools/functions.py:107: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  kde = KernelDensity(kernel=kernel, bandwidth=bandwidth).fit(y[:, None])
/home/weygoldt/Data/uni/efish/env/lib/python3.10/site-packages/gridtools/functions.py:107: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  kde = KernelDensity(kernel=kern

computed AreaUnderCurve (AUC) of KDE using sklearn.metrics.auc: 0.9955144550170782
computed AreaUnderCurve (AUC) of KDE using sklearn.metrics.auc: 0.975301153295079
computed AreaUnderCurve (AUC) of KDE using sklearn.metrics.auc: 0.9999999977347609
computed AreaUnderCurve (AUC) of KDE using sklearn.metrics.auc: 0.9999999861049931


In [8]:
plt.eventplot(raster_ts_init, linelengths=0.85,color = "black",  alpha = 0.3, zorder = 1000)
plt.eventplot(raster_ts_react, linelengths=0.85, color = "black", alpha = 0.3, zorder = 1000)